In [ ]:
"""Data inspection."""

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import numpy as np
import pickle
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
PATH = '/scratch/users/nina/output/'

In [2]:
test = pickle.load(open(f'{PATH}/training/train_losses.pkl', 'rb'))
plt.plot(test)

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/users/nina/output//training/train_losses.pkl'

In [3]:
plt.imshow(np.vstack([np.ones((20,20)),  np.ones((20,20))*color]))

NameError: name 'color' is not defined

In [4]:
foo = np.load(f'/scratch/users/johmathe/output/training/imgs/Epoch_40_recon.npy')

@interact(id=IntSlider(min=0,max=7,step=1,value=0))
def display_image(id):
    plt.imshow(foo[id][0])

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/users/johmathe/output/training/imgs/Epoch_40_recon.npy'